# Test the retrained YOLO model

Now that we have retrained our model we can test it against our 'test' images.

In [ ]:
# If you did not use the Workbench image designed for this Lab, you can uncomment and run the following line to install the required packages.
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

from ultralytics import YOLO
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt

import remote_infer

In the last notebook we saw how to re-trained the model. As this is a time-consuming tasks that would require GPUs to be efficient, we cannot really do it during the workshop.

So for your convenience, an already re-trained model, exported in the ONNX format, is available.

In [ ]:
# Get the newly trained model.

model = YOLO("https://rhods-public.s3.amazonaws.com/demo-models/ic-models/accident/accident_detect.onnx", task="detect")

In [ ]:
# Test the model against a car accident image
image_path = "images/carImage3.jpg" # We know that Image3 represents a severe accident with over 86% certainty.

_, scale, original_image = remote_infer.preprocess(image_path)

original_image: np.ndarray = cv2.imread(image_path)
blob = cv2.dnn.blobFromImage(original_image, size=(640, 640), swapRB=False)
blob = np.ascontiguousarray(blob[0].transpose((1,2,0)))
results = model.predict(blob)

In [ ]:
# Extract all the information from the result (type, bounding box, probability)

detections = []
result = results[0]
for box in result.boxes:
    class_id = int(box.cls.item())
    score = box.conf.item()
    unscaled_cords = box.xyxy.squeeze().tolist()
    cords = [round(unscaled_cords[0] * scale[1]), round(unscaled_cords[1] * scale[0]), round(unscaled_cords[2] * scale[1]), round(unscaled_cords[3] * scale[0])]
    detection = {
        'class_id': class_id,
        'class_name': result.names[class_id],
        'confidence': score,
        'box': cords,
        'scale': scale}
    detections.append(detection)
    print(detection)
    remote_infer.draw_bounding_box(original_image, class_id, score, cords[0], cords[1], cords[2], cords[3])

In [ ]:
# On the image, draw the box, the name of the class, and the probability (how much the model is sure about the detection).

img = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
fig = plt.gcf()
fig.set_size_inches(6, 3)
plt.axis('off')
plt.imshow(img)

In examining 'carImage3.jpg' the re-trainined YOLO model correctly predicts a 'car accident' with 86% confidence.  A box is then drawn around the car accident and labelled with `severe 0.86`.

Now that we have a model which can detect accident severity, let's create a predict function and serve our  model with ModelMesh.

**Go back to the Instructions of the Lab, to learn how to do that.**

**DO NOT open notebook `04-05-model-serving.ipynb` yet**.